# CH 5-6

## TOC<a id='toc'></a>
* [Ch5 Notes](#ch6_notes)
* [Ch6 Notes](#ch6_notes)

### CH5 Notes <a id='ch5_notes'></a>
[toc](#toc)

* **first-class onbject**: a program entity that can be:
    - created a runtime
    - assigned to a variable or element in a data structure
    - passed as an argument in a function
    - returned as the result of a function
* functions have attributes!
    * `my_funct.__doc__` - `help()` buitin uses this

In [1]:
def myfunct(x):
    "This is a doc"
    return x

In [2]:
myfunct.__doc__

'This is a doc'

In [3]:
help(myfunct)

Help on function myfunct in module __main__:

myfunct(x)
    This is a doc



### Higher order functions
* **higher-order functions**: a function that takes a function as an argument or returns a function as a result
    - most well known examples: *map, filter, reduce,* and *apply*
    - `apply` removed in python 3 - no longer needed
    - `map` and `filter` still built-ins in python; but their use is discouraged in favor of using listcomps and genexps
        * in python 3, these return generators - a form of iterator - so their direct substitue is genexp
    - `reduce` demoted from builtin to part of `functools` module
        * its most common use, served by `sum` builtin
        * other reducing builtins: `all()` and `any()`

### Anonymous Functions
* in python, body of lambda must be pure expressions
    - no asignment, or things like `while`, `try`, etc
* their main use is as arguments to higher order functions - otherwise they should more thank likely be regualr functions

### The Seven Flavors of Callable Objects
* call operator `()` can be used on other things beyond user defined functions
    - check if object is callable by using built-in `callable()`
* 7 flavors
    1. *User-defined*: created with `def` or `lambda`
    2. *built-in functions*: function implemented in C, like `len`
    3. *built-in methods*: method implemented in C, like `dict.get`
    4. *Methods*: functions defined in body of class
    7. *Classes*: when invoked calls `__new__` method to create instance, then `__init__` to initialize it, and returns instance
    1. *Class instance*: if class defines a `__call__` method
        - <font color=blue> This is the easiest way to create a function-like object that has internal state that must be kept accross invocations. </font>
            * other approached: decorators and closures. more on these later.
    3. *Generator functions*: functions or methods that use the `yield` key word. Returns generator object
        - more on this later

### Function introspections
* already seen `__doc__`
* they also have `__dict__`
    - can actually assign attributes to functions
* the things they have generic objects don't
    - `__call__` - impementation of () operator
    - `__code__` - metadata (argument names, arg_count, etc.) and body compiled into bytecode
    - `__globals__` - global vars of module where dunct is defined
    -  `__defaults__` - holds tuple of default values of positional and keyword arguments
    - `__kwdefaults__` - default values for *keyword-only* formal parameters
        - see below
    - `__closure__`, `__annotations__`, `__get__`, `__name__`,`__qualnmae__`

In [4]:
myfunct.__code__

<code object myfunct at 0x000002DFC7E20A50, file "<ipython-input-1-f00943e4f435>", line 1>

### parameter handling
* One of the big strength of python is its very flexible parameter handling
* combines, positional, keyword, variable of both using `*` and `**`.  - and maps arguments to formal parameters in a smart way
* *keyword-only* arguments are new in Python 3
    - name them after the argument prefixed with *
        * ex: `tag(name, *content, cls=None, **attrs)` --  `cls` is keyword only
    - will never capture unnamed positional arguments
    - can use even if don't support variable positional - put `*` by itself in signature
        * `def f(a, *, b)`
    - don't require a default value
* much of the structure of parameter handling ins encoded in `myfunc.__code__`
    - contains `myfunc.__code__.co._varnames` (includes variables defined in function)
    - contains `myfunc.__code__.co_argcount` - match the first arcount in varnames
    - `__defaults__` matched to args, backwards - a little awkard
* better way, use `inspect` module
    - `sig = inspect.signature(myfunct)` - has a lot of useful info
    - `sig.bind(...)` - binds parms to args as it would in call - useful for validation before calling
        * exposes machinery interpreter uses to bing arguments to formal parameters in function casll
        * frameworks and IDEs can use this information to validate code

In [5]:
def f(a,*,b):
    return a, b

In [6]:
f(3)

TypeError: f() missing 1 required keyword-only argument: 'b'

In [7]:
f(3,4)

TypeError: f() takes 1 positional argument but 2 were given

In [8]:
f(b=4)

TypeError: f() missing 1 required positional argument: 'a'

In [9]:
f(3,b=4)

(3, 4)

### Function Annotations
* annotation expression may be of any type. most common are classes or strings
    - ex: `def clip(text: str, max_len:'int > 0'=80) -> str:`
* no processing is done with the annotations, they are merely stored in the `__annotations__` atribute of the function (which is a dict)
* no checks performed, no enforcement, nothing - annotations have no meaning to the python interpreter. Just metadata to be used by tools like IDES, frameworks or decorators

### Packages for Functional Programming
* `operator` module
    - provide dozens of arithmentic operators that you would otherwise have to use lambdas for
        * ex: `from operator import mul; def fact(n): return reduce(mul, range(1, n+1))`
    - also provides utilities to pick items from sequences or objects: `itemgetter` and `attrgetter`
        * ex: `sorted(metro_data, key=itemgetter(1))` - replaces `lambda x: x[1]`
        * `attrgetter` is similar except extracts attributes by name (like `.`, `__getattribute__`)
            - if arg contains dot, attrgetter is smart enough to navigate nested attributes
        * for both, can pass mutliple value, and will return tuple of multiple values
        * a third similar one, but for methods, `methodcaller`
            - seems you cant pass multiple values to it though. Because others args get passed to called method
* `functools`
    * `reduce` is now poart of functools and not a builtin
    * other useful ones are `partial` and its variation `partialmethod` (used for currying - freezine args)
    * there is more, like `lru_cache`, but will discuss later

### CH6 Notes <a id='ch6_notes'></a>
[toc](#toc)

* in languages where functionas are first-class objects, some of the classic design patterns (like Strategy, visitor, command, etc.) are much easier to implement, and require less boilerplate. Can be replace by just creating different functions.
 - Strategy often implemented with flyweight design pattern - much simpler if just use defined functions.
 - tell-tale sign: single-method classes with no state
* Modules are also first-class objects! - can use this to "search your code"
    -`globals()` - return a dictionary representing current global symbol table
        * always the dictionary of the current module (if in funciton, it is module where defined not where called)
* ex: `promos = [globals()[name] for name in globals() if name.endswith('_promo') and name != 'best_promo']`

<br>
<hr>
<font color=blue> Cool using globals() to do "code introspection" - can reduce having to "wire" things manually and reduce erros due to that </font>
<hr>
<br>

* another approach - build separate module with all the `_promo` functions (except the meta-one) and use `inspect.getmembers` to get all functions:
- ex: `promos = [func for name, func in inspect.getmembers(promotions, inspect.isfunction)]`

In [2]:
import pandas as pd
import inspect

In [8]:
inspect.getmembers(pd, inspect.isfunction)[0:5]

[('Expr', <function pandas.core.computation.api.Expr(*args, **kwargs)>),
 ('Term', <function pandas.io.api.Term(*args, **kwargs)>),
 ('bdate_range',
  <function pandas.core.indexes.datetimes.bdate_range(start=None, end=None, periods=None, freq='B', tz=None, normalize=True, name=None, weekmask=None, holidays=None, closed=None, **kwargs)>),
 ('concat',
  <function pandas.core.reshape.concat.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, copy=True)>),
 ('crosstab',
  <function pandas.core.reshape.pivot.crosstab(index, columns, values=None, rownames=None, colnames=None, aggfunc=None, margins=False, margins_name='All', dropna=True, normalize=False)>)]

* a more explicit alternative for dynamically collecting promotional discount functions would be to use a simple decorator - will see this next chapter.